# Amazon Personalize AWS Exemple de personnalisation de l'utilisateur + recommandations contextuelles


## Introduction <a class="anchor" id="intro"></a>

Pour la plupart, les algorithmes d'Amazon Personalize (appelés recettes) cherchent à résoudre différentes tâches, expliquées ici :

1. **Personnalisation de l'utilisateur** – Recommande des articles en fonction de précédentes interactions de l'utilisateur avec les articles.
1. **Classement personnalisé** – Prend une collection d'articles et les ordonne dans l'ordre probable d'intérêt en utilisant une approche de type HRNN.
1. **SIMS (Éléments similaires)** – À partir d'un article donné, recommande d'autres articles avec lesquels les utilisateurs ont également interagi.

Quel que soit le cas d'utilisation, les algorithmes partagent tous une base d'apprentissage sur les données d'interaction entre l'utilisateur et l'article, qui sont définies par trois attributs fondamentaux :

1. **UserID** – L'utilisateur qui a interagi
1. **ItemID** – Article avec lequel l'utilisateur a interagi
1. **Horodatage** – L'heure à laquelle l'interaction s'est produite


## Choisir un jeu de données ou une source de données <a class="anchor" id="source"></a>
[Retour au début](#top)

Comme nous l'avons mentionné, les données d'interaction utilisateur-article sont essentielles pour commencer à utiliser le service. Cela signifie que nous devons rechercher les cas d'utilisation qui génèrent ce type de données, dont voici quelques exemples courants :

1. Applications de vidéo à la demande
1. Plateformes d'e-commerce
1. Agrégateurs/plateformes de médias sociaux

Il existe quelques lignes directrices pour définir la portée d'un problème adapté à Personalize. Nous recommandons les valeurs ci-dessous comme point de départ, bien que les [limites officielles](https://docs.aws.amazon.com/personalize/latest/dg/limits.html) soient un peu plus basses.

* Utilisateurs authentifiés
* Au moins 50 utilisateurs uniques
* Au moins 100 articles uniques
* Au moins deux douzaines d'interactions pour chaque utilisateur 

La plupart du temps, il est facile d'y parvenir, et si vous êtes dans une catégorie faible, vous pouvez souvent compenser en ayant un chiffre plus élevé dans une autre catégorie.

En règle générale, vos données n'arriveront pas sous une forme parfaite pour Personalize, et il faudra les modifier afin qu'elles soient structurées correctement. Ce bloc-notes a pour objectif de vous guider dans tout ceci. 

Pour commencer, nous allons utiliser un jeu de données d'avis sur les de compagnies aériennes. Un jeu de données créé à partir de tous les avis des utilisateurs trouvés sur Skytrax (www.airlinequality.com). Les données peuvent être consultées sur le site https://github.com/quankiquanki/skytrax-reviews-dataset 

In [274]:
import pandas as pd, numpy as np
import io
import scipy.sparse as ss
import json
import time
import datetime
import os
import sagemaker.amazon.common as smac
import boto3
import uuid
from botocore.exceptions import ClientError

### Importer et explorer votre jeu de données

In [275]:
data_dir = "airlines_data"
!mkdir $data_dir
!cd $data_dir && wget https://raw.githubusercontent.com/quankiquanki/skytrax-reviews-dataset/master/data/airline.csv


--2020-06-18 16:33:44--  https://raw.githubusercontent.com/quankiquanki/skytrax-reviews-dataset/master/data/airline.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.64.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.64.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34752262 (33M) [text/plain]
Saving to: ‘airline.csv’

airline.csv         100%[===================>]  33.14M   113MB/s    in 0.3s    

2020-06-18 16:33:45 (113 MB/s) - ‘airline.csv’ saved [34752262/34752262]



In [276]:
airline_df = pd.read_csv(data_dir + '/airline.csv')
airline_df.head()

,airline_name,link,title,author,author_country,date,content,aircraft,type_traveller,cabin_flown,route,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,wifi_connectivity_rating,value_money_rating,recommended
0,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,D Ito,Germany,2015-04-10,Outbound flight FRA/PRN A319. 2 hours 10 min f...,NaN,NaN,Economy,NaN,7.0,4.0,4.0,4.0,0.0,NaN,NaN,4.0,1
1,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Ron Kuhlmann,United States,2015-01-05,Two short hops ZRH-LJU and LJU-VIE. Very fast ...,NaN,NaN,Business Class,NaN,10.0,4.0,5.0,4.0,1.0,NaN,NaN,5.0,1
2,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,E Albin,Switzerland,2014-09-14,Flew Zurich-Ljubljana on JP365 newish CRJ900. ...,NaN,NaN,Economy,NaN,9.0,5.0,5.0,4.0,0.0,NaN,NaN,5.0,1
3,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Tercon Bojan,Singapore,2014-09-06,Adria serves this 100 min flight from Ljubljan...,NaN,NaN,Business Class,NaN,8.0,4.0,4.0,3.0,1.0,NaN,NaN,4.0,1
4,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,L James,Poland,2014-06-16,WAW-SKJ Economy. No free snacks or drinks on t...,NaN,NaN,Economy,NaN,4.0,4.0,2.0,1.0,2.0,NaN,NaN,2.0,0


Le jeu de données comporte de nombreuses colonnes que nous pouvons utiliser pour créer les jeux de données requis dans Amazon Personalize.

Nous commencerons par faire deux copies du jeu de données

In [277]:
a_interactions_df = airline_df.copy()
a_users_df = airline_df.copy()

## Création du jeu de données d’interactions

Créons le jeu de données d’interactions. En suivant les étapes suivantes :

- Abandonner les colonnes qui ne nous intéressent pas
- Créer une nouvelle colonne pour tenir compte du type d'événement
- Renommer les colonnes selon une convention d'appellation plus standard pour votre travail d'importation Amazon Personalize



In [278]:
# Keeping only 5 columns
a_interactions_df = a_interactions_df[['airline_name', 'author', 'date', 'cabin_flown', 'overall_rating']]
# Creating an additional column for Event Type
a_interactions_df['EVENT_TYPE']='RATING'
# Making sure the author name is unique without spaces
a_interactions_df['author'] = a_interactions_df['author'].str.replace(" ","")
# Rename the columns to a more Amazon Personalize standar notation
a_interactions_df.rename(columns = {'airline_name':'ITEM_ID', 'author':'USER_ID',
                              'date':'TIMESTAMP', 'cabin_flown': 'CABIN_TYPE', 'overall_rating': 'EVENT_VALUE'}, inplace = True) 
a_interactions_df.head()

,ITEM_ID,USER_ID,TIMESTAMP,CABIN_TYPE,EVENT_VALUE,EVENT_TYPE
0,adria-airways,DIto,2015-04-10,Economy,7.0,RATING
1,adria-airways,RonKuhlmann,2015-01-05,Business Class,10.0,RATING
2,adria-airways,EAlbin,2014-09-14,Economy,9.0,RATING
3,adria-airways,TerconBojan,2014-09-06,Business Class,8.0,RATING
4,adria-airways,LJames,2014-06-16,Economy,4.0,RATING


Amazon Personalize intègre des **recommandations contextuelles**, grâce auxquelles vous pouvez améliorer la pertinence des recommandations en les générant dans un contexte, par exemple le type d'appareil, le lieu, l'heure de la journée, etc. Les informations contextuelles sont également utiles à la personnalisation des utilisateurs nouveaux/non identifiés, même lorsque les interactions passées de ces utilisateurs ne sont pas connues.

Dans notre cas, nous allons utiliser le **Cabin Type (type de cabine)** comme contexte pour recommander la compagnie aérienne qui convient le mieux à notre utilisateur. Voyons quelles valeurs nous allons pouvoir transmettre comme contexte pour obtenir des recommandations


In [279]:
a_interactions_df.CABIN_TYPE.unique()

array(['Economy', 'Business Class', nan, 'Premium Economy', 'First Class'],
      dtype=object)

Comme nous pouvons le voir, la valeur actuelle de l'**horodatage** dans le jeu de données est une chaîne. Amazon Personalize exige que la valeur de l'horodatage soit de type Unix. Convertissons une valeur d'horodatage aléatoire en type Unix

In [280]:
# Get a random value from the timestamp column
arb_time_stamp = a_interactions_df.iloc[50]['TIMESTAMP']
# Transform this string to date time
date_time_obj = datetime.datetime.strptime(arb_time_stamp, '%Y-%m-%d')
print('Date:', date_time_obj.date())
# Get the date of this object
d = date_time_obj.date()
# Transform the date object to Unix time
unixtime = time.mktime(d.timetuple())
print('Unix Time: ', unixtime)

Date: 2004-03-18
Unix Time:  1079568000.0


Maintenant nous allons procéder à la même transformation pour toutes les valeurs de la colonne horodatage

In [281]:
# Define a function
def convert_to_unix(string_date):
    date_time_obj = datetime.datetime.strptime(string_date, '%Y-%m-%d')
    d = date_time_obj.date()
    return time.mktime(d.timetuple())

# Apply this function across the Timestamp column
a_interactions_df['TIMESTAMP'] = a_interactions_df['TIMESTAMP'].apply(convert_to_unix)
a_interactions_df.head(5)

,ITEM_ID,USER_ID,TIMESTAMP,CABIN_TYPE,EVENT_VALUE,EVENT_TYPE
0,adria-airways,DIto,1.428624e+09,Economy,7.0,RATING
1,adria-airways,RonKuhlmann,1.420416e+09,Business Class,10.0,RATING
2,adria-airways,EAlbin,1.410653e+09,Economy,9.0,RATING
3,adria-airways,TerconBojan,1.409962e+09,Business Class,8.0,RATING
4,adria-airways,LJames,1.402877e+09,Economy,4.0,RATING


Observons certaines des propriétés de notre jeu de données

In [282]:
a_interactions_df.describe()

,TIMESTAMP,EVENT_VALUE
count,4.139600e+04,36861.000000
mean,1.373950e+09,6.039527
std,5.771909e+07,3.214680
min,0.000000e+00,1.000000
25%,1.350864e+09,3.000000
50%,1.389658e+09,7.000000
75%,1.412122e+09,9.000000
max,1.438474e+09,10.000000


Y a-t-il des valeurs Null ?

In [283]:
a_interactions_df.isnull().values.any()

True

Oublions ces valeurs Null et vérifions qu'il n'y en a pas

In [284]:
a_interactions_df = a_interactions_df.dropna()
a_interactions_df.isnull().values.any()

False

Maintenant que nous nos données sont prêtes pour Amazon Personalize, sauvegardons-les dans un fichier local

In [285]:
interactions_filename = "a_interactions.csv"
a_interactions_df.to_csv((data_dir + "/"+interactions_filename), index=False, float_format='%.0f')

## Création du jeu de données d’utilisateurs

Créons le jeu de données d’utilisateurs. En suivant les étapes suivantes :

- Abandonner les colonnes qui ne nous intéressent pas
- Créer une nouvelle colonne tenant compte de la nationalité comme métadonnées utilisateur
- Renommer les colonnes selon une convention d'appellation plus standard pour votre travail d'importation Amazon Personalize


In [286]:
# Copy the complete airlines data set
a_users_df = airline_df.copy()
# Select only interested columns
a_users_df = a_users_df[['author', 'author_country']]
# Clean up the authors string
a_users_df['author'] = a_users_df['author'].str.replace(" ","")
# Rename the columns
a_users_df.rename(columns = { 'author':'USER_ID', 'author_country':'NATIONALITY'}, inplace = True) 
# Drop any null values
a_users_df = a_users_df.dropna()
# Save your file locally
users_filename = "a_users.csv"
a_users_df.to_csv((data_dir +"/"+users_filename), index=False)

## Configurer un compartiment S3 et un rôle IAM <a class="anchor" id="bucket_role"></a>
[Retour au début](#top)

Pour l'instant, nous avons téléchargé, manipulé et enregistré les données sur l'instance Amazon EBS associée à l'instance qui exécute ce bloc-notes Jupyter. Toutefois, Amazon Personalize aura besoin d'un compartiment S3 pour servir de source à vos données, ainsi que de rôles IAM pour accéder à ce compartiment. Mettons tout cela en place.

Utilisez les métadonnées stockées sur l'instance sous-jacente de ce bloc-notes Amazon SageMaker pour déterminer la région dans laquelle il fonctionne. Si vous utilisez un bloc-notes Jupyter en dehors d'Amazon SageMaker, définissez simplement la région sous forme de chaîne comme suit. Le compartiment Amazon S3 doit se trouver dans la même région que les ressources Amazon Personalize que nous avons créées jusqu'à présent.

In [288]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


Les noms des compartiments Amazon S3 sont uniques au niveau mondial. Pour créer un nom de compartiment unique, le code ci-dessous ajoutera la chaîne de caractères `personalizepoc` à votre numéro de compte AWS. Puis il crée un compartiment avec ce nom dans la région découverte dans la cellule précédente.

In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
suffix = str(np.random.uniform())[4:9]
bucket_name = "personalize-user-personalization-example" + suffix
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

### Charger les données vers S3

Maintenant que votre compartiment Amazon S3 a été créé, chargez le fichier CSV de nos données d'interaction utilisateur-article. 

In [294]:
interactions_filename = data_dir + '/a_interactions.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

In [295]:
user_metadata_file = data_dir + '/a_users.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(user_metadata_file).upload_file(user_metadata_file)

## Créer des groupes de jeux de données et le jeu de données d'interactions <a class="anchor" id="group_dataset"></a>
[Retour au début](#top)

Le plus haut niveau d'isolation et d'abstraction avec Amazon Personalize est le *groupe de jeux de données*. Toute information stockée dans l'un de ces groupes de jeux de données n'a aucun impact sur un autre groupe de données ou sur les modèles créés à partir de celui-ci – ils sont entièrement indépendants. Vous pouvez ainsi réaliser de nombreuses expériences, et c'est en partie grâce à cela que vos modèles restent privés et ne sont entraînés que sur vos données. 

Avant d'importer les données préparées antérieurement, il faut créer un groupe de jeux de données et y ajouter un jeu de données qui gère les interactions.

Les groupes de jeux de données peuvent comporter les types d'informations suivants :

* Interactions utilisateur-article
* Flux d'événements (interactions en temps réel)
* Métadonnées de l'utilisateur
* Métadonnées de l'article

Avant de créer le groupe de jeux de données et le jeu de données pour nos données d'interaction, vérifions que votre environnement peut communiquer avec Amazon Personalize.

In [293]:
personalize = boto3.client(service_name='personalize')
personalize_runtime = boto3.client(service_name='personalize-runtime')
personalize_events = boto3.client(service_name='personalize-events')

### Créer un groupe de jeux de données

La cellule suivante va créer un nouveau groupe de jeux de données avec le nom *airlines-dataset-group* + un suffixe

In [64]:
dataset_group_name = "airlines-dataset-group-" + suffix

create_dataset_group_response = personalize.create_dataset_group(
    name = dataset_group_name
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:144386903708:dataset-group/airlines-dataset-group-55035",
  "ResponseMetadata": {
    "RequestId": "a8bb75fb-f15b-45da-997e-08eb14d7733a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:14:12 GMT",
      "x-amzn-requestid": "a8bb75fb-f15b-45da-997e-08eb14d7733a",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Pour pouvoir utiliser le groupe de jeux de données, il doit être actif. Cela peut prendre une ou deux minutes. Exécutez la cellule ci-dessous et attendez qu'elle affiche le statut ACTIF. Il vérifie l'état du groupe de jeux de données toutes les secondes, jusqu'à un maximum de 3 heures.

In [65]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


Maintenant que vous avez un groupe de jeux de données, vous pouvez créer un jeu de données pour les données d'interaction.

# Créer des jeux de données

### Jeu de données d’interactions

Tout d'abord, définissez un schéma afin d'indiquer à Amazon Personalize le type de jeu de données que vous chargez. Plusieurs mots-clés réservés et obligatoires sont requis dans le schéma, en fonction du type de jeu de données. Des informations plus détaillées peuvent être retrouvées dans la [documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Ici, vous allez créer un schéma pour les données d'interactions, qui nécessite les champs `USER_ID`, `ITEM_ID`, `TIMESTAMP`, `CABIN_TYPE`, `EVENT_TYPE`, `EVENT_VALUE` et `TIMESTAMP`. Ils doivent être définis dans le schéma en respectant l'ordre dans lequel ils apparaissent dans le jeu de données.

In [55]:
schema_name="airlines-interaction-schema-"+suffix

In [56]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name":"CABIN_TYPE",
            "type": "string",
            "categorical": True
        },
        {
          "name": "EVENT_TYPE",
          "type": "string"
        },
        {
          "name": "EVENT_VALUE",
          "type": "float"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = schema_name,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/airlines-interaction-schema-55035",
  "ResponseMetadata": {
    "RequestId": "4e045a61-d479-485c-93ff-6072076ccaa9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:10:34 GMT",
      "x-amzn-requestid": "4e045a61-d479-485c-93ff-6072076ccaa9",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [66]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn,
    name = "airlines-dataset-interactions-" + suffix
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:144386903708:dataset/airlines-dataset-group-55035/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "968e6cac-310a-4889-8243-e86ef90696ed",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:15:14 GMT",
      "x-amzn-requestid": "968e6cac-310a-4889-8243-e86ef90696ed",
      "content-length": "109",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Jeu de données des utilisateurs

Ici, vous allez créer un schéma pour les données d'utilisateurs, qui nécessite les champs `USER_ID`, et `NATIONALITY`. Ils doivent être définis dans le schéma en respectant l'ordre dans lequel ils apparaissent dans le jeu de données.


In [62]:
metadata_schema_name="airlines-users-schema-"+suffix

In [63]:
metadata_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "NATIONALITY",
            "type": "string",
            "categorical": True
        }
    ],
    "version": "1.0"
}

create_metadata_schema_response = personalize.create_schema(
    name = metadata_schema_name,
    schema = json.dumps(metadata_schema)
)

metadata_schema_arn = create_metadata_schema_response['schemaArn']
print(json.dumps(create_metadata_schema_response, indent=2))


{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/airlines-users-schema-55035",
  "ResponseMetadata": {
    "RequestId": "17844e2f-860d-484a-bb39-ab4a10e7b9fd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:13:50 GMT",
      "x-amzn-requestid": "17844e2f-860d-484a-bb39-ab4a10e7b9fd",
      "content-length": "93",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
dataset_type = "USERS"
create_metadata_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = metadata_schema_arn,
    name = "airlines-metadata-dataset-users-" + suffix
)

metadata_dataset_arn = create_metadata_dataset_response['datasetArn']
print(json.dumps(create_metadata_dataset_response, indent=2))

## Configurer un compartiment S3 et un rôle IAM <a class="anchor" id="bucket_role"></a>

Pour l'instant, nous avons téléchargé, manipulé et enregistré les données sur l'instance Amazon EBS associée à l'instance qui exécute ce bloc-notes Jupyter. Toutefois, Amazon Personalize aura besoin d'un compartiment S3 pour servir de source à vos données, ainsi que de rôles IAM pour accéder à ce compartiment. Mettons tout cela en place.

Utilisez les métadonnées stockées sur l'instance sous-jacente de ce bloc-notes Amazon SageMaker pour déterminer la région dans laquelle il fonctionne. Si vous utilisez un bloc-notes Jupyter en dehors d'Amazon SageMaker, définissez simplement la région sous forme de chaîne comme suit. Le compartiment Amazon S3 doit se trouver dans la même région que les ressources Amazon Personalize que nous avons créées jusqu'à présent.

### Définir la politique du compartiment S3
Amazon Personalize doit pouvoir lire le contenu de votre compartiment S3. Il faut donc ajouter une politique de compartiment qui l'autorise.

In [69]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy));

### Créer un rôle IAM

Amazon Personalize doit pouvoir assumer des rôles dans AWS, afin de disposer des autorisations nécessaires pour exécuter certaines tâches. Créons un rôle IAM et ajoutons-lui les politiques requises. Le code ci-dessous associe des politiques très permissives ; veuillez utiliser des politiques plus restrictives pour toute application de production.

In [70]:
iam = boto3.client("iam")

role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise
        
# sometimes need to wait a bit for the role to be created
time.sleep(45)
print(role_arn)

arn:aws:iam::144386903708:role/PersonalizeS3Role-55035


# Créer vos tâches d'importation de jeux de données

## Importer les données d'interactions <a class="anchor" id="import"></a>

Précédemment, vous avez créé le groupe de jeux de données et le jeu de données pour héberger vos informations. Vous allez maintenant exécuter une tâche d'importation qui chargera les données du compartiment S3 dans le jeu de données Amazon Personalize. 

In [109]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "airlines-dataset-import-job-"+suffix,
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/airlines-dataset-import-job-14078",
  "ResponseMetadata": {
    "RequestId": "d118cf11-b568-4767-99d5-30a15871981a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:46:38 GMT",
      "x-amzn-requestid": "d118cf11-b568-4767-99d5-30a15871981a",
      "content-length": "121",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Importer les données des utilisateurs <a class="anchor" id="import"></a>

Précédemment, vous avez créé le groupe de jeux de données et le jeu de données pour héberger vos informations. Vous allez maintenant exécuter une tâche d'importation qui chargera les données du compartiment S3 dans le jeu de données Amazon Personalize. 

In [110]:
create_metadata_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "airlines-users-metadata-dataset-import-job-"+suffix,
    datasetArn = metadata_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, user_metadata_file)
    },
    roleArn = role_arn
)

metadata_dataset_import_job_arn = create_metadata_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_metadata_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/airlines-users-metadata-dataset-import-job-14078",
  "ResponseMetadata": {
    "RequestId": "679d9401-568d-45e0-ba8c-df8f1574228d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:46:40 GMT",
      "x-amzn-requestid": "679d9401-568d-45e0-ba8c-df8f1574228d",
      "content-length": "136",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Attendre que les tâches d'importation de jeux de données aient le statut ACTIF

Pour pouvoir utiliser le jeu de données, la tâche d'importation doit être active. Exécutez la cellule ci-dessous et attendez qu'elle affiche le statut ACTIF. Il vérifie l'état de la tâche d'importation toutes les secondes, jusqu'à un maximum de 3 heures.

L'importation des données peut prendre un certain temps, en fonction de la taille du jeu de données. Dans cette démo, la tâche d'importation de données a été préexécutée.

In [111]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


In [112]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = metadata_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


Lorsque l'importation du jeu de données est active, vous êtes prêt à commencer à créer des modèles avec la recette de personnalisation de l'utilisateur AWS.

## Créer des solutions <a class="anchor" id="solutions"></a>
[Retour au début](#top)

Dans ce bloc-notes, nous allons créer une solution avec la recette suivante :

1. aws-user-personalization


Dans Amazon Personalize, une variation spécifique d'un algorithme est appelée une recette. Des recettes différentes conviennent à des situations différentes. Un modèle entraîné s'appelle une solution, et chaque solution peut avoir de nombreuses versions qui se rapportent à un volume de données spécifique au moment où le modèle a été entraîné.

Tout d'abord, nous allons énumérer toutes les recettes qui sont prises en charge. Cela vous permettra d'en sélectionner une et de l'utiliser pour créer votre modèle.

In [113]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims
arn:aws:personalize:::recipe/aws-user-personalization


Le résultat est juste une représentation JSON de tous les algorithmes mentionnés dans l'introduction.

Ensuite, nous sélectionnerons des recettes spécifiques et créerons des modèles avec elles.

### Personnalisation de l'utilisateur AWS

La personnalisation de l'utilisateur AWS est l'un des modèles de recommandation les plus avancés disponibles. Elle permet de mettre à jour les recommandations en temps réel en fonction du comportement des utilisateurs. Elle tend également à surpasser d'autres approches, comme le filtrage collaboratif. Cette recette est la plus longue à entraîner. Commençons par elle.

Dans notre cas d'utilisation, à l'aide des données d'avis sur les compagnies aériennes, nous pouvons utiliser la personnalisation de l'utilisateur AWS pour recommander des compagnies aériennes à un utilisateur en fonction de son comportement antérieur en matière de marquage des artistes. Rappelez-vous que nous avons utilisé les données de marquage pour représenter les interactions positives entre un utilisateur et un artiste.

Tout d'abord, sélectionnez la recette en trouvant l'ARN dans la liste des recettes ci-dessus.

In [114]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

#### Créer la solution

Tout d'abord, vous créez une solution à l'aide de la recette. Même si vous fournissez le jeu de données ARN dans cette étape, le modèle n'est pas encore entraîné. Considérez cela comme un identifiant plutôt qu'un modèle entraîné.

Notez que nous avons activé le HPO ici. Il s'agit d'une bonne idée lorsque 


In [ ]:
create_solution_response = personalize.create_solution(
    name = "airlines-user-personalization-solution-HPO-"+suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
    performHPO=True
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### Créer la version de solution

Une fois que vous avez une solution, vous devez créer une version afin d'achever l'entraînement du modèle. L'entraînement peut prendre un certain temps, plus de 25 minutes, et une moyenne de 40 minutes pour cette recette avec notre jeu de données. Normalement, nous utiliserions une boucle while pour interroger le système jusqu'à ce que la tâche soit terminée. Toutefois, cette tâche bloquerait l'exécution d'autres cellules, et l'objectif ici est de créer de nombreux modèles et de les déployer rapidement. Nous allons donc mettre en place la boucle while pour toutes les solutions plus bas dans ce bloc-notes. Vous y trouverez également des instructions pour visualiser la progression dans la console AWS.

In [117]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/airlines-hrnn-metadata-solution-HPO-14078/54a6c563",
  "ResponseMetadata": {
    "RequestId": "148a0fbd-5465-4619-ac90-449c0ef23b73",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 22:01:32 GMT",
      "x-amzn-requestid": "148a0fbd-5465-4619-ac90-449c0ef23b73",
      "content-length": "127",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [118]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

## Évaluer les versions de la solution <a class="anchor" id="eval"></a>
[Retour au début](#top)

Il ne devrait pas falloir plus d'une heure pour entraîner toutes les solutions de ce bloc-notes. Pendant l'entraînement, nous vous recommandons de prendre le temps de lire en détail les différents algorithmes (recettes) et leur comportement. Le moment est également propice pour envisager des alternatives à la manière dont les données ont été introduites dans le système et le type de résultats que vous attendez.

Lorsque la création des solutions est terminée, l'étape suivante est consacrée à l'obtention des métriques d'évaluation. Personalize calcule ces métriques sur la base d'un sous-ensemble de données d'entraînement. L'image ci-dessous illustre comment Personalize répartit les données. Pour 10 utilisateurs, avec 10 interactions chacun (un cercle représente une interaction), les interactions sont classées de la plus ancienne à la plus récente sur la base de l'horodatage. Personalize utilise toutes les données d'interaction de 90 % des utilisateurs (cercles bleus) pour entraîner la version de la solution, et les 10 % restants pour l'évaluation. Pour chacun des utilisateurs dans les 10 % restants, 90 % de leurs données d'interaction (cercles verts) sont utilisées comme entrée pour le recours au modèle entraîné. Les 10 % restants de leurs données (cercle orange) sont comparés à la solution produite par le modèle et sont utilisés pour calculer les métriques d'évaluation.

![métriques Personalize](static/imgs/personalize_metrics.png)

Nous recommandons de lire [la documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) pour comprendre les métriques, mais nous avons également copié des parties de la documentation ci-dessous pour plus de commodité.

Vous devez comprendre les termes suivants à propos de l'évaluation dans Personalize :

* *Recommandation pertinente* désigne une recommandation qui correspond à une valeur dans les données de test pour l'utilisateur particulier.
* *Rang* désigne la position d'un article recommandé dans la liste des recommandations. La position 1 (haut de la liste) est présumée être la plus pertinente pour l'utilisateur.
* *Requête* désigne l'équivalent interne de l'appel GetRecommendations.

Les métriques fournies par Personalize sont les suivantes :
- **coverage** : La proportion d'articles uniques recommandés à partir de toutes les requêtes sur le nombre total d'articles uniques dans les données de formation (comprend à la fois les jeux de données d'articles et d'interactions).
- **mean_reciprocal_rank_at_25** : La [moyenne des rangs réciproques](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) de la première recommandation pertinente parmi les 25 premières recommandations pour toutes les requêtes. Cette métrique est appropriée si vous êtes intéressé par la recommandation unique la mieux classée.
- **normalized_discounted_cumulative_gain_at_K** : Le gain actualisé suppose que les recommandations situées plus bas dans une liste de recommandations sont moins pertinentes que les recommandations situées plus haut. Par conséquent, chaque recommandation est réduite (pondération plus faible) par un facteur dépendant de sa position. Pour produire le [gain cumulé actualisé](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) à K, toutes les recommandations actualisées pertinentes dans les K premières recommandations sont additionnées. Le gain cumulé actualisé normalisé (NDCG) est le DCG divisé par le DCG idéal de sorte que le NDCG soit compris entre 0 et 1. (Le DCG idéal est celui où les K premières recommandations sont triées par pertinence). Amazon Personalize utilise un facteur de pondération de 1/log(1 + position), où le haut de la liste est la position 1. Cette métrique récompense les articles pertinents qui apparaissent près du haut de la liste, car le haut d'une liste suscite généralement plus d'attention.
- **precision_at_K** : Le nombre de recommandations pertinentes parmi les K premières recommandations divisé par K. Cette métrique récompense la recommandation précise des articles pertinents.

Examinons les métriques d'évaluation de chacune des solutions produites dans ce bloc-notes. *Veuillez noter que vos résultats peuvent varier par rapport aux résultats décrits dans ce bloc-notes, en raison de la qualité du jeu de données LastFM.* 

### Métriques de personnalisation de l'utilisateur AWS

Tout d'abord, récupérez les métriques d'évaluation pour la version de la solution de personnalisation de l'utilisateur AWS.

In [120]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))


{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/airlines-hrnn-metadata-solution-HPO-14078/54a6c563",
  "metrics": {
    "coverage": 0.4046,
    "mean_reciprocal_rank_at_25": 0.2035,
    "normalized_discounted_cumulative_gain_at_10": 0.2909,
    "normalized_discounted_cumulative_gain_at_25": 0.3174,
    "normalized_discounted_cumulative_gain_at_5": 0.2418,
    "precision_at_10": 0.0444,
    "precision_at_25": 0.022,
    "precision_at_5": 0.0605
  },
  "ResponseMetadata": {
    "RequestId": "156a6e70-152b-4940-8b17-048252419fd0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 23:02:24 GMT",
      "x-amzn-requestid": "156a6e70-152b-4940-8b17-048252419fd0",
      "content-length": "424",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


# Créer une campagne à partir de la solution

## Créer des campagnes <a class="anchor" id="create"></a>

Une campagne est une version de la solution hébergée, un point de terminaison que vous pouvez interroger pour obtenir des recommandations. La tarification est fixée en estimant la capacité de débit (demandes de personnalisation de l'utilisateur par seconde). Lors du déploiement d'une campagne, vous définissez une valeur minimale de débit par seconde (TPS). Ce service, comme beaucoup d'autres au sein d'AWS, évoluera automatiquement en fonction de la demande. Néanmoins, si la latence est stratégique, vous voudrez peut-être prendre des dispositions pour une demande plus importante. Pour cette POC et cette démo, tous les seuils de débit minimum sont définis à 1. Pour plus d'informations, reportez-vous à la [page de tarification](https://aws.amazon.com/personalize/pricing/).

Commençons à déployer les campagnes.

### Personnalisation de l'utilisateur AWS

Déployez une campagne pour votre version de la solution de personnalisation de l'utilisateur AWS. Le déploiement d'une campagne peut prendre environ 10 minutes. Normalement, nous utiliserions une boucle while pour interroger le système jusqu'à ce que la tâche soit terminée. Cependant, la tâche bloquerait l'exécution d'autres cellules, et le but ici est de créer de multiples campagnes. Nous allons donc mettre en place la boucle while pour toutes les campagnes plus bas dans ce bloc-notes. Vous y trouverez également des instructions pour visualiser la progression dans la console AWS.

In [122]:
create_campaign_response = personalize.create_campaign(
    name = "airlines-metadata-campaign-"+suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 2,    
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:144386903708:campaign/airlines-metadata-campaign-14078",
  "ResponseMetadata": {
    "RequestId": "4c882630-86aa-4c5d-accd-75225f9804a4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 23:25:37 GMT",
      "x-amzn-requestid": "4c882630-86aa-4c5d-accd-75225f9804a4",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [123]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


### Personnalisation de l'utilisateur AWS

La personnalisation de l'utilisateur AWS est l'un des algorithmes les plus avancés fournis par Amazon Personalize. Il prend en charge la personnalisation des articles pour un utilisateur spécifique sur la base de son comportement passé et peut prendre en compte les événements en temps réel afin de modifier les recommandations pour un utilisateur sans nouvel entraînement. 

Puisque l'algorithme de personnalisation de l'utilisateur AWS repose sur un échantillon d'utilisateurs, chargeons les données dont nous avons besoin pour cela et sélectionnons trois utilisateurs aléatoires.

In [262]:
users_df = pd.read_csv(data_dir + '/a_users.csv')
# Render some sample data
users_df.sample(5)

,USER_ID,NATIONALITY
3918,JHartley,United Kingdom
27477,DDriscoll,United Kingdom
19563,BrianElliott,United Kingdom
22989,AHornbuckle,Australia
35724,CMoon,United Kingdom


Nous présentons ci-après les recommandations pour nos trois utilisateurs aléatoires. Nous explorerons ensuite les interactions en temps réel avant de passer au classement personnalisé.

Nous créons à nouveau une fonction d'aide pour présenter les résultats dans un tableau de données bien structuré.

#### Résultats des appels d'API

In [165]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    
#   Context Recommendations
    context_options = ['None','Economy', 'Business Class','Premium Economy', 'First Class']
    
    for context in context_options:
        # Get the recommendations
        if context=='none':
            get_recommendations_response = personalize_runtime.get_recommendations(
                campaignArn = campaign_arn,
                userId = str(user_id),
            )
        else:
            get_recommendations_response = personalize_runtime.get_recommendations(
                campaignArn = campaign_arn,
                userId = str(user_id),
                context = {
                  'CABIN_TYPE': context
                }
            )
        # Build a new dataframe of recommendations
        item_list = get_recommendations_response['itemList']
        recommendation_list = []
        for item in item_list:
            recommendation_list.append(item['itemId'])
    #     print(recommendation_list)
        new_rec_DF = pd.DataFrame(recommendation_list, columns = [context])
        # Add this dataframe to the old one
        recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [263]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

      USER_ID     NATIONALITY
37013    RDow  United Kingdom


,None,Economy,Business Class,Premium Economy,First Class
0,british-airways,thomson-airways,british-airways,thomson-airways,united-airlines
1,united-airlines,thomas-cook-airlines,virgin-atlantic-airways,virgin-atlantic-airways,british-airways
2,thomson-airways,united-airlines,turkish-airlines,air-new-zealand,american-airlines
3,virgin-atlantic-airways,easyjet,united-airlines,british-airways,china-southern-airlines
4,air-new-zealand,monarch-airlines,china-southern-airlines,united-airlines,delta-air-lines
5,turkish-airlines,virgin-atlantic-airways,qatar-airways,thomas-cook-airlines,lufthansa
6,china-southern-airlines,british-airways,emirates,turkish-airlines,alaska-airlines
7,thomas-cook-airlines,jet2-com,air-france,monarch-airlines,virgin-atlantic-airways
8,lufthansa,lufthansa,american-airlines,china-southern-airlines,us-airways
9,american-airlines,american-airlines,lufthansa,eva-air,thomson-airways


In [264]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

      USER_ID NATIONALITY
26198   CJeff   Singapore


,None,Economy,Business Class,Premium Economy,First Class
0,philippine-airlines,philippine-airlines,cathay-pacific-airways,cathay-pacific-airways,singapore-airlines
1,cathay-pacific-airways,singapore-airlines,philippine-airlines,air-france,thai-airways
2,singapore-airlines,tigerair,malaysia-airlines,eva-air,philippine-airlines
3,ana-all-nippon-airways,jetstar-asia,srilankan-airlines,air-new-zealand,delta-air-lines
4,thai-airways,cathay-pacific-airways,thai-airways,philippine-airlines,emirates
5,air-india,airasia,singapore-airlines,klm-royal-dutch-airlines,ana-all-nippon-airways
6,china-eastern-airlines,cebu-pacific,air-india,airasia-x,china-southern-airlines
7,dragonair,scoot,emirates,qantas-airways,alaska-airlines
8,srilankan-airlines,ana-all-nippon-airways,ana-all-nippon-airways,china-southern-airlines,american-airlines
9,air-france,dragonair,asiana-airlines,united-airlines,china-eastern-airlines


Ici, nous voyons clairement que les recommandations sont différentes pour chaque utilisateur. Si vous aviez besoin d'un cache pour ces résultats, vous pourriez commencer par exécuter les appels d'API par tous vos utilisateurs et stocker les résultats. Vous pourriez aussi utiliser une exportation par lot. Cette approche qui sera abordée plus tard dans ce bloc-notes.

La prochaine rubrique concerne les événements en temps réel. Personalize a la capacité de suivre les événements de votre application afin de mettre à jour les recommandations présentées à l'utilisateur. Cela est particulièrement utile pour les charges de travail liées aux médias, comme la vidéo à la demande, où l'intention du client peut différer selon qu'il regarde le film avec ses enfants ou seul.

En outre, les événements enregistrés par l’intermédiaire de ce système sont stockés jusqu'à ce que vous émettiez un ordre de suppression, et ils sont utilisés comme données historiques avec les autres données d'interaction que vous avez fournies lors de la formation de vos prochains modèles.

#### Événements en temps réel

Commencez par créer un suivi d'événement qui est attaché à la campagne.

In [150]:
response = personalize.create_event_tracker(
    name='AirlinesEventsTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:144386903708:event-tracker/d2e7ccdc
820029aa-b00c-4eff-9e6f-60830bb68508


Nous allons créer un code qui simule l'interaction d'un utilisateur avec un article particulier. Après avoir exécuté ce code, vous obtiendrez des recommandations qui diffèrent des résultats ci-dessus.

Nous commençons par créer quelques méthodes pour la simulation d'événements en temps réel.

In [200]:
session_dict = {}

def send_user_rating(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
        "itemId": str(ITEM_ID),
        "eventValue": 10,
        "cabinType": "Economy"
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
        trackingId = TRACKING_ID,
        userId= str(USER_ID),
        sessionId = session_ID,
        eventList = [{
            'sentAt': int(time.time()),
            'eventType': 'RATING',
            'properties': event_json
            }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id):
    # Interact with the airline
    # Sending a rating of 10 in Economy class for the airline with that user
    send_user_rating(USER_ID=user_id, ITEM_ID=item_id)
    
    
    #   Context Recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
        context = {
          'CABIN_TYPE': 'Economy'
        }
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        recommendation_list.append(item['itemId'])
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [item_id+'|Economy'])
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df


À ce stade, nous n'avons pas encore généré d'événements en temps réel ; nous avons seulement configuré le code. Pour comparer les recommandations formulées avant et après les événements en temps réel, nous allons choisir un utilisateur et générer les recommandations originales pour lui.

## Recommandations avant d'utiliser le Traqueur d'événements

In [265]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)
user_id = users[0]
recommendations_df_users

        USER_ID NATIONALITY
6313  ACrociani       Italy


,None,Economy,Business Class,Premium Economy,First Class
0,british-airways,alitalia,iberia,british-airways,american-airlines
1,alitalia,brussels-airlines,british-airways,virgin-atlantic-airways,british-airways
2,iberia,ryanair,qatar-airways,united-airlines,delta-air-lines
3,brussels-airlines,easyjet,alitalia,brussels-airlines,united-airlines
4,qatar-airways,iberia,brussels-airlines,turkish-airlines,lufthansa
5,lufthansa,aer-lingus,emirates,alitalia,emirates
6,american-airlines,aegean-airlines,lufthansa,air-france,qatar-airways
7,united-airlines,lufthansa,turkish-airlines,lufthansa,swiss-international-air-lines
8,icelandair,qatar-airways,swiss-international-air-lines,icelandair,us-airways
9,delta-air-lines,tap-portugal,oman-air,delta-air-lines,iberia


In [266]:
user_id

'ACrociani'

Nous disposons à présent d'une liste de recommandations pour cet utilisateur avant d'avoir appliqué des événements en temps réel. Choisissons à présent 3 artistes au hasard avec lesquels nous allons simuler l'interaction de notre utilisateur et, voyons comment cela modifie les recommandations.

In [267]:
# Next generate 3 random Airlines
airlines = a_interactions_df.sample(3)['ITEM_ID'].tolist()
airlines

['thai-airways', 'austrian-airlines', 'united-airlines']

In [268]:
user_recommendations_df = pd.DataFrame()
# Note this will take about 15 seconds to complete due to the sleeps
for airline in airlines:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, airline)
    time.sleep(5)
print(user_id)
user_recommendations_df

ACrociani


,thai-airways|Economy,austrian-airlines|Economy,united-airlines|Economy
0,alitalia,ryanair,brussels-airlines
1,brussels-airlines,brussels-airlines,lufthansa
2,ryanair,easyjet,turkish-airlines
3,easyjet,tap-portugal,austrian-airlines
4,iberia,aegean-airlines,tap-portugal
5,aer-lingus,turkish-airlines,alitalia
6,aegean-airlines,lufthansa,germanwings
7,lufthansa,alitalia,aegean-airlines
8,qatar-airways,iberia,swiss-international-air-lines
9,tap-portugal,air-india,air-berlin


Dans la cellule ci-dessus, la première colonne après l'index est constituée des recommandations par défaut de l'utilisateur provenant du modèle de personnalisation de l'utilisateur AWS, et chaque colonne suivante comporte un en-tête des compagnies aériennes avec lesquelles l'utilisateur a interagi par l’intermédiaire d’un événement en temps réel, ainsi que les recommandations après cet événement. 

Il est possible que le comportement ne change pas beaucoup. Cela est dû à la nature relativement limitée de ce jeu de données. Pour mieux comprendre ce phénomène, essayez de simuler la notation de compagnies aériennes aléatoires avec des notations aléatoires, et vous devriez constater un impact plus prononcé.